## Analytical solution of multi-spore diffusion

The general solution for the concentration on a 3D lattice with an initial delta pulse at $(x,y,z)=(0,0,0)$ of size $c_0\times V_s$, where $c_0$ is the initial concentration in the spore and $V_s$ is the volume of the source, follows the Green's function:

$$
\begin{equation}
c(x,y,z,t)=G(x,y,z,t)=\frac{c_0V_s}{(4\pi Dt)^{3/2}}e^{-\frac{\Delta{x}^2+\Delta{y}^2+\Delta{z}^2}{4Dt}}
\end{equation}
$$

Or, alternatively, defining $r=\sqrt{\Delta{x}^2+\Delta{y}^2+\Delta{z}^2}$ as the distance from the measured point to the source:

$$
\begin{equation}
c(r,t)=G(r,t)=\frac{c_0V_s}{(4\pi Dt)^{3/2}}e^{-\frac{r^2}{4Dt}}
\end{equation}
$$

It is assumed for now that the spores are small compared to the scale of the observed domain and are a negligable obstacle for diffusion. However, they do add new concentration to the system, at a rate determined by the permeation through their cell wall. If this rate is denoted as $Q$, the concentration at a point in space incorporates the accumulation of solute added to the system over time:

$$
\begin{equation}
c(r,t)=\int_0^t{Q{(t')}G{(r, t-t')}dt'}
\end{equation}
$$

The inhibitor release rate at time $t$ can be expressed as

$$
\begin{equation}
Q{(t)}=JA,
\end{equation}
$$

where $J$ is the flux through the cell wall and $A$ is the spore surface area. The flux is defined as

$$
\begin{equation}
J=P_s\Delta{c_s{(t)}},
\end{equation}
$$

with $P_s$ being the permeation constant (measured in units of velocity) and $\Delta{c_s{(t)}}$ the concentration difference between the spore and the exterior. This concentration difference evolves from an initial difference $\Delta{c_s{(0)}}$ as follows:

$$
\begin{equation}
\Delta{c_s{(t)}}=\Delta{c_s{(0)}}e^{-t/\tau},
\end{equation}
$$

where $\tau=\frac{V_s}{AP_s}$. The initial concentration drop can be thought of as the difference between the concentration at the interior of the spore interface and the concentration immediately outside of it. Following the previous assumption about the spore neglected as an obstacle, the location just outside of it is the entry point for new concentration, namely $(x_0, y_0, z_0)$. Therefore:

$$
\begin{equation}
\Delta{c_s{(0)}}=c_0-c(x_0,y_0,z_0,0)
\end{equation}
$$

But since no prior inhibitor is assumed just outside the spore at $t=0$, this simplifies to

$$
\begin{equation}
\Delta{c_s{(0)}}=c_0
\end{equation}
$$

Thus, the concentration transport at time $t$ amounts to

$$
\begin{equation}
Q{(t)}=AP_sc_0e^{-\frac{P_sAt}{V_s}}.
\end{equation}
$$

Plugging this equation into the expression for the concentration and substituting the respective Green's function yields:

$$
\begin{equation}
c(r,t)=\int_0^t{AP_sc_0e^{-\frac{P_sAt'}{V_s}}\frac{c_0V_s}{(4\pi D(t-t'))^{3/2}}e^{-\frac{r^2}{4D(t-t')}}dt'}
\end{equation}
$$

The integral can be simplified by defining $\tau'=t-t'$, such that $t'=t-\tau'$ and $dt'=-d\tau'$:

$$
\begin{equation}
c(r,t)=AP_s\frac{c_0^2V_s}{(4\pi D)^{3/2}}\int_0^t{\frac{1}{\tau'^{3/2}}\exp{\left(-\frac{P_sA(t-\tau')}{V_s}-\frac{r^2}{4D\tau}\right)}d\tau'}
\end{equation}
$$

$$
\begin{equation}
c(r,t)=\frac{AV_sP_sc_0^2}{(4\pi D)^{3/2}}e^{-\frac{P_sAt}{V_s}}\int_0^t{\tau'^{-3/2}\exp{\left(-\frac{P_sA\tau'}{V_s}-\frac{r^2}{4D\tau}\right)}d\tau'}
\end{equation}
$$

Now, assuming that there are $M$ sources which start releasing simultaneously with the same $c_0$, the resulting concentration over the lattice is simply the sum of the contributions from each source:

$$
\begin{equation}
c(x,y,z,t)=\frac{AV_sP_sc_0^2}{(4\pi D)^{3/2}}e^{-\frac{P_sAt}{V_s}}\sum_{i=0}^{M}{\int_0^t{\tau'^{-3/2}\exp{\left(-\frac{P_sA\tau'}{V_s}-\frac{r_i^2}{4D\tau}\right)}d\tau'}},
\end{equation}
$$

where $r_i=\sqrt{\Delta{x_i}^2+\Delta{y_i}^2+\Delta{z_i}^2}$ is the respective distance to a single source with index $i$.

In [1]:
import numpy as np

In [35]:
def concentration_from_permeating_sources(x, t, x_sources, c0, D, Ps, A, V, dt=0.01):
    """
    Compute the concentration at a given point and time
    due to singular permeating sources. The integral is 
    computed using the trapezoidal rule.
    inputs:
        x (numpy array): 3D positions of the observation point
        t (float): time of observation
        x_sources (numpy array): 3D positions of the sources
        c0 (float): initial concentration at the sources
        D (float): diffusion coefficient
        Ps (float): source release rate
        A (float): source area
        V (float): source volume
        dt (float): time step size
    """

    prefactor = A * Ps * V * c0 ** 2 / np.power(4 * np.pi * D, 3/2) * np.exp(-Ps*A*t/V)
    src_sum = 0

    # Iterate over sources
    for x_i in x_sources:
        print(f"Computing source at {x_i}")
        # Compute the integral over time using the trapezoidal rule
        t_vals = np.arange(dt, t+dt, dt)
        # print(t_vals)
        # print(t)
        G = np.zeros((t_vals.shape[0], x.shape[0]))
        for i, tau in enumerate(t_vals):
            # print(x.shape)
            # print(x_i[np.newaxis, :].shape)
            # print(np.linalg.norm(x - x_i[np.newaxis, :]).shape)
            exp = np.exp(- Ps * A * tau / V - np.linalg.norm(x - x_i[np.newaxis, :], axis=1)**2 / (4 * D * tau)) * dt
            # print(exp.shape)
            # print(G.shape)
            # print(G[i].shape)
            G[i] = exp / np.sqrt(tau ** 3)
        print(G.shape)
        print(t_vals.shape)
        src_sum += np.trapz(G, t_vals[:, np.newaxis], axis=0)
    
    return prefactor * src_sum

# Inputs
N = 64  # Number of grid points
dx = 5  # Grid spacing in microns
D = 600  # micron^2/s
Ps = 0.0002675353069511818 # microns/s
# x_obs = np.zeros((N, N, N))  # Observation positions
x_sources = np.array([[N // 4, N // 4, N // 4], [3*N // 4, 3*N // 4, 3*N // 4]]) * dx  # Source positions
x_obs = np.vstack([x_sources, np.array([N // 2, N // 2, N // 2])]) * dx  # Observation positions
print(x_obs)
c0 = 1.018  # Initial concentration at sources in M
V = 125 # microns^3
A = 150 # microns^2

# Compute the concentration at each observation point
t_max = 14400  # Maximum time in seconds
ts = np.linspace(1, t_max, 4)  # Time points
concentrations_analytical = np.zeros((ts.shape[0], x_obs.shape[0]))
for i, t in enumerate(ts):
    print(f"Computing concentration at time {t}")
    concentrations_analytical[i] = concentration_from_permeating_sources(x_obs, t, x_sources, c0, D, Ps, A, V)
print(concentrations_analytical)

[[ 400  400  400]
 [1200 1200 1200]
 [ 160  160  160]]
Computing concentration at time 1.0
Computing source at [80 80 80]
(100, 3)
(100,)
Computing source at [240 240 240]
(100, 3)
(100,)
Computing concentration at time 4800.666666666667
Computing source at [80 80 80]
(480067, 3)
(480067,)
Computing source at [240 240 240]
(480067, 3)
(480067,)
Computing concentration at time 9600.333333333334
Computing source at [80 80 80]
(960034, 3)
(960034,)
Computing source at [240 240 240]
(960034, 3)
(960034,)
Computing concentration at time 14400.0
Computing source at [80 80 80]
(1440000, 3)
(1440000,)
Computing source at [240 240 240]
(1440000, 3)
(1440000,)
[[3.11900070e-23 0.00000000e+00 6.30267312e-12]
 [6.08669644e-09 4.15519306e-10 1.92154340e-08]
 [1.31122002e-09 9.50909251e-11 4.12334126e-09]
 [2.80995035e-10 2.04976014e-11 8.83321496e-10]]
